In [44]:
import pandas as pd
import numpy as np

In [59]:
fbrain_connectivity = pd.read_parquet('Connectivity_783.parquet')
fbrain_neurons = pd.read_csv('2023_03_23_completeness_630_final.csv')

In [73]:
print(fbrain_neurons.index[-1])
print(np.unique(fbrain_connectivity[['Presynaptic_ID','Postsynaptic_ID']].values).shape[0])

127399
138639


In [76]:
fbrain_neurons['Completed'].sum()

127400

In [78]:
fbrain_neurons

,Unnamed: 0,Completed
0,720575940596125868,True
1,720575940597856265,True
2,720575940597944841,True
3,720575940598267657,True
4,720575940599333574,True
...,...,...
127395,720575940660862849,True
127396,720575940660863105,True
127397,720575940660865409,True
127398,720575940660868481,True


Transform MANC dataset into df with columns: ['Presynaptic_ID', 'Postsynaptic_ID', 'Presynaptic_Index', 'Postsynaptic_Index', 'Connectivity', 'Excitatory', 'Excitatory x Connectivity']
save as .parquet file (possible with pandas)



In [14]:
neuron_info = pd.read_csv('mystuff/neurons.csv')
connections = pd.read_csv('mystuff/fetch_traced_adjacencies_roi_conn.csv')

C:\Users\bschult1\AppData\Local\Temp\ipykernel_12004\374839653.py:1: DtypeWarning: Columns (11,13,15,22,23,35,37,40,44,46) have mixed types. Specify dtype option on import or set low_memory=False.
  neuron_info = pd.read_csv('mystuff/neurons.csv')


In [53]:
# Create a dict to assign a unique index to each neuron (key: bodyId)
index_dict = { key:val for key,val in zip(neuron_info['bodyId'], neuron_info.index) }
# Create a dict stating the synapse sign for each neuron (key: bodyId) 
synsign_dict = {}
for key in index_dict.keys():
    nt = neuron_info.loc[neuron_info['bodyId']==key, 'predictedNt'].values

    if nt == 'glutamate' or nt == 'gaba':
        synsign_dict[key] = -1
    else:
        synsign_dict[key] = 1


pre_index = []
post_index = []
excitatory = []
# For each connection save the pre and postsynaptic neuron index and the synapse sign
for i in connections.index:
    pre_index.append(index_dict[connections.loc[i, 'bodyId_pre']])
    post_index.append(index_dict[connections.loc[i, 'bodyId_post']])
    excitatory.append(synsign_dict[connections.loc[i, 'bodyId_pre']])

pre_index = np.array(pre_index)
post_index = np.array(post_index)
excitatory = np.array(excitatory)

In the paper, did they group/sum all connections between two neurons of the same IDs or did they not? 
    - check the paper as well as the code

In [83]:
# Create a DataFrame analog to the one used for the brain modelling (columns: Presynaptic_ID, Postsynaptic_ID, Presynaptic_Index, Postsynaptic_Index, Connectivity, Excitatory, Excitatory x Connectivity)
df_dict = {'Presynaptic_ID':connections['bodyId_pre'].values, 'Postsynaptic_ID':connections['bodyId_post'].values, 'Presynaptic_Index':pre_index, 'Postsynaptic_Index':post_index, 'Connectivity':connections['weight'].values, 
           'Excitatory':excitatory, 'Excitatory x Connectivity':excitatory*connections['weight'].values}

# Make data frame
manc_df = pd.DataFrame(df_dict)

# Save to parquet
manc_df.to_parquet('./manc_connectivity.parquet')

# Save list of neurons as csv
manc_completeness = pd.DataFrame({'':neuron_info['bodyId'].values, 'Completed':np.repeat(True, neuron_info['bodyId'].size)})
manc_completeness.to_csv('./manc_completeness.csv')